In [3]:
language = 'french'
setting = "Post-apocalypse zombie"
first_message = "Tu te trouves au cœur de ce qui était autrefois une ville prospère, aujourd'hui réduite à un chaos silencieux appelé Ville Fantôme. Autour de toi, les bâtiments délabrés témoignent d’une époque passée tandis que les ombres mouvantes des zombies hantent chaque coin sombre. Tu es Alexandre Durand, un ingénieur civil transformé en survivant aguerri depuis le début de cette pandémie terrifiante. Ta conviction reste intacte : trouver un moyen de stopper définitivement ces créatures infectées est non seulement ta mission personnelle, mais aussi celle de tous ceux qui espèrent voir renaître une forme quelconque de normalité. La route sera longue et périlleuse, mais tu sais désormais que chaque détail peut faire la différence entre la survie et l’extinction. Que décides-tu de faire ?"

In [4]:
from typing import Optional

In [5]:
from langchain_community.llms.mlx_pipeline import MLXPipeline
from langchain_community.chat_models.mlx import ChatMLX
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.globals import set_verbose
from langchain.globals import set_debug

#set_debug(True)
#set_verbose(True)

print('Loading model...')
# Load model from huggingface, using the MLX framework to take advantage of Apple Silicon
llm = MLXPipeline.from_model_id(
    "mlx-community/Meta-Llama-3.1-8B-Instruct-8bit",
    #"mlx-community/Meta-Llama-3.1-8B-Instruct-bf16",
    #"mlx-community/Qwen2.5-32B-Instruct-4bit", # Let's try using a larger model, see if that improves results
    #"mlx-community/Llama-3.2-3B-Instruct-8bit",
    pipeline_kwargs={"max_tokens": 2048, "temp": 0.2, "repetition_penalty":1.2},
)

predictable_llm = MLXPipeline.from_model_id(
    "mlx-community/Meta-Llama-3.1-8B-Instruct-8bit",
    pipeline_kwargs={"max_tokens": 2048, "temp": 0, "repetition_penalty":1.2},
)

# Setup verbose mode: https://stackoverflow.com/a/77629872/10914628
model = ChatMLX(llm=llm, verbose=True).with_config({'callbacks': [ConsoleCallbackHandler()]})

predictable_model = ChatMLX(llm=predictable_llm)
print('Model loaded.')



Loading model...


/Users/frieder/Documents/GitHub/procedural-stories/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 105993.24it/s]


Model loaded.


In [6]:
from owlready2 import *

onto = get_ontology('file://story_poptest.rdf').load()

# Verify that the story world is internally coherent, at least by OWL standards
#with onto:
    #sync_reasoner() 

    # for l in onto.CurrentLocation.instances(): print(f"CurrentLocation: {l.hasName}")
    # for c in onto.Character.instances(): print(c.hasName)

    # print(onto.Player.instances()[0].INDIRECT_isLocatedAt)

Issue: The reasoner will use the state of the ontology at the moment it is instantiated.
If the player moves to a new location, the reasoner will not be updated with this new information.

Source: https://dice-group.github.io/owlapy/usage/reasoning_details.html

Solution: Use isolated worlds, one that is clean, which we update and change with asserted facts, the second one that is used for inference.
When the player moves, we change the clean world, and use it for inference.


In [7]:
from urllib.parse import quote, unquote

def encode_entity_name(name: str) -> str:
    return quote(name.lower())

def decode_entity_name(encoded_name: str) -> str:
    return unquote(encoded_name)

In [8]:
from Levenshtein import distance

def find_levenshtein_match(string: str, entity_list: list, threshold: int = 5):
    """Find a match amongst a list of entities from the ontology, in case the LLM made slight typos !"""
    minimum = threshold
    closest_entity = None
    for entity in entity_list:
        dist = distance(string, entity.hasName)
        if dist < minimum:
            minimum = dist
            closest_entity = entity
    return closest_entity

# Conversation loop
So, we have pre-generated the first AI message, which lays out the start of the game.

Then, all subsequent turns will use the same system prompt, as well as dynamically enhancing the player's message with relevant information from the KG

After the player sends their message, we either can:

- Have the LLM respond without knowledge, then analyze the message pair for changes to the KG
- Have the LLM analyze the user's message for intent (especially moving to new locations, which will require the information of the new location to generate the response)

I think the second approach is just better, especially in the case of moving to new locations, which change the knowledge necessary to generate the response.



Once the AI response is generated, we also want to extract changes to the world from the message pair, based on the AI response (item lost, item gained)


So... Basically we need to do intent analysis of the Human message.


Let's summarize a typical conversation turn:

0. The player reads the game's message and decides on something to do.
1. The player's message is analyzed with a prompt, made to extract actionable intent that requires new knowledge (move to other location)
2. If the player moves to another location, we need to retrieve the information about this new location and use that info to generate the AI response. At the same time, we update the KG (player's position)
3. Generate the game response
4. While the player reads and decides on their next turn and types it, analyze the last (message, response) pair for changes to the world: updates to the player's inventory (remove, add or change item's description), updates to characters.

## Move intent
Immediately after the player's message, we need to know if we need to load information about a different location, in order to guide the generation of the response.

In [9]:
INTENT_ANALYSIS_TEMPLATE = """
Human: Please analyze the player's message and determine if they **explicitly** intend to move to a nearby location **immediately**.
Nearby locations: {{nearby_locations_names}}
Player message: "{{player_message}}"

**Please output only the *exact location name* or 'none' with ABSOLUTELY no additional explanation, notes or details. If it doesn't fit, output 'none'**
Output:
"""

In [29]:
from langchain_core.prompts import PromptTemplate

def extract_move_intent(model, message: str, onto) -> Optional[onto.Location]:
    prompt = PromptTemplate(
        template=INTENT_ANALYSIS_TEMPLATE,
        template_format='jinja2'
    )

    with onto:
        nearby_locations = onto.Player.instances()[0].INDIRECT_isLocatedAt.INDIRECT_isLinkedToLocation
        
        nearby_locations_names = []
        for l in nearby_locations:
            if l.name == "CurrentLocation":
                continue
            
            nearby_locations_names.append(l.hasName)

        #print(nearby_locations_names)
        
        # print(prompt.invoke({
        #     'nearby_locations_names': ", ".join(f'"{l}"' for l in nearby_locations_names),
        #     'player_message': message,
        #     'language': language
        # }).text)
        
        chain = prompt | model
        
        analysis = chain.invoke({
            'nearby_locations_names': ", ".join(f'"{l}"' for l in nearby_locations_names),
            'player_message': message,
            'language': language
        })
        #print(f"Output: {analysis.content}")
        
        
        result = analysis.content.strip().strip('"')

        if result.lower() == 'none':
            return None
        
        #print(result)
        
        entity = find_levenshtein_match(result, onto.Location.instances())
        
        return entity


In [31]:
print(extract_move_intent(predictable_model,'Aller à la mine', onto))
print(extract_move_intent(predictable_model,'Aller au bunker', onto))
print(extract_move_intent(predictable_model,'Manger une pomme', onto))


None
story_poptest.bunker%20souterrain
None


## Response generation

In [12]:
CHAT_SYSTEM_PROMPT = """
You are an LLM designed to act as the engine for a text adventure game set in "{{setting}}".

Keep in mind that more things can and should be revealed later, after interaction with the player, so feel free to keep some information to yourself for later.
Keep in mind that not everything that is in a location is necessarily immediately visible or known to the player. Things can be hidden within a location, and a location can be quite large.

The game is played by interactions between the game (you) and the player. The player will type in natural language whatever they want to do, etc.
Do not reveal everything all at once: let the player discover things. Only output natural text, as one would read in a book they actually were the hero of.

Your messages should be short. Please do not produce lengthy messages. Your messages should be one to two sentences long. The player can always ask for more details ! For dialogues, you will output only one line of dialogue, and let the player respond.

Use the game elements provided.

Always answer in {{language}}
"""

In [45]:
HUMAN_MESSAGE_TEMPLATE = """
[INST]
# History
{{history}}

# Game state
The player's current location is "{{location.hasName}}". {{location.hasName}} is described as "{{location.hasDescription}}".

The locations accessible from where the player is are {{nearby_locations_names}}. You should discreetly tell the player they can go there, without being too explicit.

The player's character is named "{{player.hasName}}" and is described as "{{player.hasDescription}}".
The player's goal is "{{player.hasGoal.hasDescription}}"

{%- if characters_nearby %}
Characters present in {{location.hasName}}:
{%- for character in characters_nearby %}
    - {{ character.hasName }}: {{character.hasDescription}} (narrative importance {{character.hasImportance}})
{%- endfor %}
{%- else %}
    There are no other characters present in {{location.hasName}}.
{%- endif %}

{%- if items_nearby %}
Items present in {{location.hasName}}:
{%- for item in items_nearby %}
    - {{ item.hasName }}: {{item.hasDescription}} (narrative importance {{item.hasImportance}})
{%- endfor %}
{%- else %}
    There are no items present in {{location.hasName}}.
{%- endif %}

{%- if player.INDIRECT_ownsItem %}
Items in the player's inventory:
{%- for item in player.INDIRECT_ownsItem %}
    - {{ item.hasName }}: {{item.hasDescription}} (narrative importance {{item.hasImportance}})
{%- endfor %}
{%- else %}
    The player has no items in their inventory.
{%- endif %}

{%- if move_intent_location %}
The parser has detected that the player intends to move to {{move_intent_location.hasName}}.
    {{move_intent_location.hasName}} is described as "{{move_intent_location.hasDescription}}".
    {%- if move_intent_location.INDIRECT_containsCharacter %}
        Characters present in {{move_intent_location.hasName}}:
        {%- for character in move_intent_location.INDIRECT_containsCharacter %}
            - {{ character.hasName }}: {{character.hasDescription}} (narrative importance {{character.hasImportance}})
        {%- endfor %}
    {%- endif %}
    {%- if move_intent_location.INDIRECT_containsItem %}
        Items present in {{move_intent_location.hasName}}:
        {%- for item in move_intent_location.INDIRECT_containsItem %}
            - {{ item.hasName }}: {{item.hasDescription}} (narrative importance {{item.hasImportance}})
        {%- endfor %}
    {%- endif %}

If you confirm the move, add the token "<CONFIRM_MOVE>" to your response.
**Keep in mind that the player will read your response before typing theirs, and only the token will be removed from your response.**
{%- endif %}

Answer directly and briefly to the user's message. **You answer should be one or two sentences at most !** If the player is curious they will ask.
The player cannot invent new items, locations or characters. You cannot invent new locations or characters, except for sub-locations of the current location. You have to always refer to the given information above.
Do not reveal all the given information at once.
[/INST]
The player's message:
{{message}}
"""

### Create a conversation history

In [46]:
from langchain_core.messages import BaseMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel,Field
from typing import List

# Define a simple in memory history
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    k: int = Field(default=12)
    messages: List[BaseMessage] = Field(default_factory=list)

    system_prompt: str = Field(default=CHAT_SYSTEM_PROMPT)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)
    
    def get_history(self) -> List[BaseMessage]:
        # Dynamically render the history: return only the last k messages, with the system prompt first
        messages = [('system', self.system_prompt)]
        messages.extend(self.messages[-self.k:])
        return messages
    
    def get_history_prompt(self) -> str:
        history_prompt = ChatPromptTemplate.from_messages(
            self.get_history(),
            template_format='jinja2'
        )

        return history_prompt.format()

    def clear(self) -> None:
        self.messages = []

# input example for the ChatPromptTemplate
# {
#     "conversation": [ # <--- this is the output of the InMemoryHistory
#         ("human", "Hi!"),
#         ("ai", "How can I assist you today?"),
#         ("human", "Can you make me an ice cream sundae?"),
#         ("ai", "No.")
#     ]
# }

In [53]:

history = InMemoryHistory()

# Render the system prompt
system_prompt_template = PromptTemplate(template=CHAT_SYSTEM_PROMPT, template_format='jinja2')
history.system_prompt = system_prompt_template.invoke({
    'setting': setting,
    'language': language
}).text

# Add the first messages to the history
history.add_messages([('ai', first_message)])

In [54]:

async def converse(message: str):
    #history.add_messages([('human', message)])
    # 1. Extract move intent
    move_intent_location = extract_move_intent(predictable_model, message, onto)

    # 2. Retrieve relevant information from the KG
    # 2.1 If the player intends to move, add the new location information to the chat prompt
    if move_intent_location:
        print(f'Move intent: {move_intent_location}')
        # Add the move intent and new location information to the chat prompt


    # 2.2 Retrieve relevant information from the KG
    with onto:
        player = list(onto.Player.instances())[0]
        current_location = player.INDIRECT_isLocatedAt
        characters_nearby = current_location.INDIRECT_containsCharacter
        locations_nearby = current_location.INDIRECT_isLinkedToLocation
        nearby_locations_names = [l.hasName for l in locations_nearby]
        items_nearby = current_location.INDIRECT_containsItem

        print(f'Player is at {current_location}')
        print(f'Characters nearby: {characters_nearby}')
        print(f'Locations nearby: {nearby_locations_names}')
        print(f'Items nearby: {items_nearby}')



    chat_prompt = PromptTemplate(
        template=HUMAN_MESSAGE_TEMPLATE,
        template_format='jinja2'
    )

    chain = chat_prompt | model

    response = await chain.ainvoke(
        {
            "setting": setting, 
            "language": language,
            "location": current_location,
            "move_intent_location": move_intent_location,
            'characters_nearby': characters_nearby,
            'items_nearby': items_nearby,
            'player': player,
            'nearby_locations_names': nearby_locations_names,
            "message": message,
            "history": history.get_history_prompt()
        },
    )

    # Find if the LLM accepted the move, and filter out the <CONFIRM_MOVE> token
    if '<CONFIRM_MOVE>' in response.content:
        print('Move confirmed by the LLM')
        response = response.replace('<CONFIRM_MOVE>', '').strip()

        # If the LLM accepted the move, update the KG
        with onto:
            for follower in player.INDIRECT_hasFollower:
                follower.isLocatedAt = move_intent_location
            player.isLocatedAt = move_intent_location
    
    # Add the message pair to the history
    history.add_messages([
            ('human', message),
            ('ai', response.content)
        ])
    
    return response.content

In [55]:
set_verbose(False)
# Test the conversation loop
while True:
    message = input('Player: ')
    print(f'Player: {message}')

    # Exit the loop
    if message == 'exit':
        break

    print(await converse(message))


Player: Ou suis-je ?


Parent run 256bb882-5afa-4b77-8f9e-7e5de1ecd781 not found for run 5f036eca-5e58-41d9-bb9c-5df4d10cbc99. Treating as a root run.


Player is at story_poptest.ville%20fant%C3%B4me
Characters nearby: [story_poptest.alexandre%20durand, story_poptest.lucas%20valois]
Locations nearby: ['Forêt Délaissée', 'Bunker Souterrain']
Items nearby: [story_poptest.couteau%20survie%20multi-fonctions, story_poptest.carte%20routi%C3%A8re%20ancienne]
System: 
You are an LLM designed to act as the engine for a text adventure game set in "Post-apocalypse zombie".

Keep in mind that more things can and should be revealed later, after interaction with the player, so feel free to keep some information to yourself for later.
Keep in mind that not everything that is in a location is necessarily immediately visible or known to the player. Things can be hidden within a location, and a location can be quite large.

The game is played by interactions between the game (you) and the player. The player will type in natural language whatever they want to do, etc.
Do not reveal everything all at once: let the player discover things. Only output natu

Parent run 14160a12-0841-4478-923f-96aca3d2d16d not found for run a3507d87-b93d-42c6-a086-a98af4107998. Treating as a root run.


Player is at story_poptest.ville%20fant%C3%B4me
Characters nearby: [story_poptest.alexandre%20durand, story_poptest.lucas%20valois]
Locations nearby: ['Forêt Délaissée', 'Bunker Souterrain']
Items nearby: [story_poptest.couteau%20survie%20multi-fonctions, story_poptest.carte%20routi%C3%A8re%20ancienne]
System: 
You are an LLM designed to act as the engine for a text adventure game set in "Post-apocalypse zombie".

Keep in mind that more things can and should be revealed later, after interaction with the player, so feel free to keep some information to yourself for later.
Keep in mind that not everything that is in a location is necessarily immediately visible or known to the player. Things can be hidden within a location, and a location can be quite large.

The game is played by interactions between the game (you) and the player. The player will type in natural language whatever they want to do, etc.
Do not reveal everything all at once: let the player discover things. Only output natu

Parent run 9e602050-bf9c-41d3-beef-f6025eab1207 not found for run ae7eb3ff-5f22-4a3e-9232-a5dd0f6cc0dd. Treating as a root run.


Player is at story_poptest.ville%20fant%C3%B4me
Characters nearby: [story_poptest.alexandre%20durand, story_poptest.lucas%20valois]
Locations nearby: ['Forêt Délaissée', 'Bunker Souterrain']
Items nearby: [story_poptest.couteau%20survie%20multi-fonctions, story_poptest.carte%20routi%C3%A8re%20ancienne]
System: 
You are an LLM designed to act as the engine for a text adventure game set in "Post-apocalypse zombie".

Keep in mind that more things can and should be revealed later, after interaction with the player, so feel free to keep some information to yourself for later.
Keep in mind that not everything that is in a location is necessarily immediately visible or known to the player. Things can be hidden within a location, and a location can be quite large.

The game is played by interactions between the game (you) and the player. The player will type in natural language whatever they want to do, etc.
Do not reveal everything all at once: let the player discover things. Only output natu

Langchain is complex and abstract, so there isn't a component really well suited for our needs.
I tried making RunnableWithMessageHistory work, but it was too much work for too little gain.
So we just an in memory history to store the chat, and define our chain manually.

This is the attempt at using RunnableWithMessageHistory:

In [16]:
# https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html
from typing import Optional, List

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableFieldSpec,
    RunnablePassthrough,
)
from langchain_core.runnables.history import RunnableWithMessageHistory


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []


# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]


In [ ]:

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", CHAT_SYSTEM_PROMPT),
    ('ai', first_message),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{{message}}"),
], template_format='jinja2')

chain = chat_prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key="message",
    history_messages_key="history",
)


# Retrieve relevant information from the KG
with onto:
    player = list(onto.Player.instances())[0]
    current_location = player.INDIRECT_isAtLocation
    characters_nearby = current_location.INDIRECT_containsCharacter
    locations_nearby = current_location.INDIRECT_isLinkedToLocation
    nearby_locations_names = [l.hasName for l in locations_nearby]
    items_nearby = current_location.INDIRECT_containsItem


    print(chain_with_history.invoke(  # noqa: T201
        {"setting": setting, 
        "language": language, 
        'characters_nearby': characters_nearby,
        'items_nearby': items_nearby,
        'player': player,
        'nearby_locations_names': nearby_locations_names,
        "message": "Aller à la forge"},
        config={"configurable": {"session_id": "foo"}}
    ).content)

# Uses the store defined in the example above.
print(store)  # noqa: T201

# print(chain_with_history.invoke(  # noqa: T201
#     {"setting": setting, "question": "What's its inverse"},
#     config={"configurable": {"session_id": "foo"}}
# ))

# print(store)  # noqa: T201

In [ ]:
player_message = "Thora, je compte partir conquerrir des terres. Va tu m'apporter ton soutien ?"

with onto:
    player = list(onto.Player.instances())[0]
    current_location = player.INDIRECT_isAtLocation
    characters_nearby = current_location.INDIRECT_containsCharacter
    locations_nearby = current_location.INDIRECT_isLinkedToLocation
    nearby_locations_names = [l.hasName for l in locations_nearby]
    items_nearby = current_location.INDIRECT_containsItem


    print(chain_with_history.invoke(  # noqa: T201
        {"setting": setting, 
        "language": language, 
        'characters_nearby': characters_nearby,
        'items_nearby': items_nearby,
        'player': player,
        'nearby_locations_names': nearby_locations_names,
        "message": player_message},
        config={"configurable": {"session_id": "foo"}}
    ).content)

After the response, we need to report any change to the world state into the knowledge graph.

A player needs to be able to interact with the graph in a few ways:
- move to a nearby location
- claim ownership (of an unowned item): take, buy
- change ownership (of an owned item): give, drop, sell
- change in emotional state of any character
- change in relationship between characters
- update item description (for example if an item deteriorates, etc)
- update character description (for example if a character needs to remember something)
- update location description (for example if the player changed the location in some notable way)
- trigger event
- update events memory (?)
- death of a character (?)

But these are a lot of tasks to ask from one prompt to a small model like Llama3.1-8B-8bit

We can classify these tasks in groups:
- Inventory related tasks
- Character related tasks
- Location related tasks

Now, we can imagine using a prompt for each one of these categories, in order to get a fair balance of speed and accuracy

In [ ]:
LOCATION_ACTIONS_PARSER_TEMPLATE = """
Human: Please analyze the player's message and determine if they intend to interact with a location.

**Action Types**:
- "move" if the player wants to go to a nearby location immediately.
- "inspect" if the player wants to examine or observe the current location.
- "modify" if the player intends to alter or change the location in some way.

**Nearby locations**: {{nearby_locations_names}}
Player message: "{{player_message}}"
Game response: "{{game_response}}"

**Please output one of the following:**
- For **move**: output the exact name of the nearby location.
- For **inspect**: output "inspect".
- For **modify**: output "modify".
- If none of these fit, output "none".

Output:
"""

In [ ]:
INVENTORY_ACTIONS_PARSER_TEMPLATE = """
Human: Please analyze the player's message and determine if they intend to interact with an item.

**Action Types**:
- "claim" if the player wants to take or pick up an unowned item.
- "give" if the player intends to give an item they own to another character.
- "drop" if the player intends to leave an item they own in the current location.
- "destroy" if an item 

**Owned items**: {{player_owned_items}}
**Unowned items in the location**: {{unowned_items}}

Player message: "{{player_message}}"
Game response: "{{game_response}}"

**Please output one of the following:**
- For **claim**: output "take: [item_name]" (e.g., "take: map").
- For **give**: output "give: [item_name]" (e.g., "give: coin").
- For **drop**: output "drop: [item_name]" (e.g., "drop: torch").
- If none of these fit, output "none".

Output:
"""


In [ ]:
CHARACTER_ACTIONS_PARSER_TEMPLATE = """
Human: Please analyze the player's message and corresponding game response to determine if the interaction changed something about the characters present in the scene

**Action Types**:.
- "relationship" if the relationship between two characters has changed

**Characters present**: {{characters_present}}
Player message: "{{player_message}}"
Game response: "{{game_response}}"

**Please output one of the following:**
- For **relationship**: output "relationship: [character_name]" (e.g., "relationship: Scrooge Mcduck").
- If none of these fit, output "none".

Output:
"""